In [30]:
import pandas as pd
import numpy as np
import pygmo as pg
import random
import functools
import math as m
import copy
from numpy import genfromtxt

# Geometry modules
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

# Plotting libraries
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.patches as patches 
import seaborn as sns

# Misc
import time 
from time import process_time
from scipy.spatial import distance_matrix
from scipy.spatial import ConvexHull
import secrets

# skopt
from skopt.space import Real, Categorical, Integer
from skopt import BayesSearchCV, space, plots
from skopt.utils import use_named_args
from skopt.plots import plot_convergence
from skopt import gp_minimize

# sklearn modules
from sklearn.metrics import accuracy_score, mean_squared_error, roc_auc_score, roc_curve
from sklearn.metrics import RocCurveDisplay, precision_recall_curve, recall_score
from sklearn.metrics import make_scorer, precision_score, classification_report
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.inspection import permutation_importance
from sklearn import datasets
from sklearn.svm import SVC
from sklearn.compose import ColumnTransformer
from sklearn.metrics import make_scorer
from sklearn.linear_model import LogisticRegressionCV

from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.naive_bayes import GaussianNB 

from deap import algorithms, base, creator, tools


# For ignoring warnings about bad classifiers - it's the nature of the algorithm to come across these
import warnings
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.simplefilter("ignore", category=RuntimeWarning)

# Matplotlib configuration
%matplotlib inline
plt.rcParams['figure.figsize'] = (8,6)
plt.rcParams['figure.dpi'] = 80
plt.rc('axes', titlesize=12)
plt.rc('axes', labelsize=12)

matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'

## Declare Datasets

In [31]:
data_set = 'churn'

In [32]:
X_df = pd.read_csv(f'UCI/{data_set}/X_train.csv')

X_train = pd.read_csv(f'UCI/{data_set}/X_train.csv').values
X_test = pd.read_csv(f'UCI/{data_set}/X_test.csv').values
y_train = genfromtxt(f'UCI/{data_set}/y_train.csv', delimiter=',').astype(int)
y_test = genfromtxt(f'UCI/{data_set}/y_test.csv', delimiter=',').astype(int)

In [33]:
# Declare standard scaler
scaler = StandardScaler()
# Transform training and testing data
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [34]:
# Number of features in model
total_features = int(X_train.shape[1])

# Feature names
feat_names = np.asarray(list(X_df.columns))

# Number of iterations (for all MOEAs in this workbook)
num_iters = 20

## Declaring classifier to run all MOEAs on

In [36]:
# Set of learning algorithms
model_dict = {
    "SVC lin": SVC(random_state=42,kernel='linear',max_iter=int(1e4), probability=True),
    "Log lasso": LogisticRegression(solver='liblinear',random_state=42,penalty='l1'),
    "Log ridge": LogisticRegression(random_state=42,penalty='l2'),
    "Gauss NB": GaussianNB(), # Naive Bayes
    "Dec trees": DecisionTreeClassifier(random_state=42),
    "Grad boost": GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=42),
    "SVC rbf": SVC(random_state=42,kernel='rbf', max_iter=int(1e4)),
    "Random forest": RandomForestClassifier(random_state=42)
}

In [37]:
model_string = 'Log lasso'
clf_ = model_dict[model_string]

## Objectives

In [38]:
# Declare the classification error measure - 'accuracy' or 'recall'
metric = 'recall'

In [39]:
objective = 'num_features'
# objective = 'time'

if objective == 'time':
    comp_time_all_X = get_model_time(model_string)
    print(comp_time_all_X)

## Declare time or number of features as objective

In [40]:
def get_model_time(model_string):
    
    '''
    Returns the seconds of how long a classifier takes to fit the whole data matrix
    '''
    
    pipeline = Pipeline([
        ("model", model_dict[model_string])
    ])
    
    times = []
    
    # Times are inconsistent - get max by looping through several times
    for _ in range(100):
    
        start = process_time()
        pipeline.fit(X_train, y_train)
        end = process_time()
        
        times.append(end-start)
    
    return np.mean(times)

In [41]:
# Declare path to save to for all outputs
if objective == 'time':
    path = 'Other_MOEAs_time'
else:
    path = 'Other_MOEAs'

In [42]:
def normalise(value):
    '''
    I: an evaluation value
    O: its normalised value 
    '''

    # The worst possible scenario is using all features (max comp time) 
    nadir = comp_time_all_X
    # The best possible scenario is a negligible time
    ideal = 1e-8
                    
    return ((value - ideal) / (nadir - ideal))

def normalise(value):
    return value

## Getting Non-Dom Fronts

In [43]:
def get_nondom_fronts(data):
    
    ndf, dl, dc, ndr = pg.fast_non_dominated_sorting(data[:,[1,2]])
    
    best = data[ndf[0]]
    
    # Sort by ascending
    best_sorted = best[best[:, 0].argsort()[::-1]]
    
    uniques = np.unique([tuple(row) for row in best_sorted], axis=0)
    
    uniques = uniques[uniques[:, 0].argsort()[::-1]]
    
    return uniques

# NSGA-II

In [44]:
nsga_all_evals = np.zeros((1, 3 + total_features))

def evaluate_individual(individual):
    
    selected_features = [bool(bit) for bit in individual]

    X_train_selected = X_train[:, selected_features]
    X_test_selected = X_test[:, selected_features]    
    n_feats = len(np.where(np.asarray(individual) == 1)[0].tolist())
    
    # Catch error of passing an array of zeros to classifier
    if n_feats == 0:
        return 0,0
        
    start = process_time()
    model = clf_.fit(X_train_selected, y_train)
    end = process_time()
    y_pred = model.predict(X_test_selected)
    
    if metric == 'accuracy':
        score = accuracy_score(y_test, y_pred)
    else:
        score = recall_score(y_test, y_pred)
        
    if objective == 'time':
        obj2 = normalise(end-start)
    else:
        obj2 = n_feats

    this_eval = np.zeros((1, 3 + total_features))
    this_eval[:,:3] = (score, obj2, (1-score))
    this_eval[:,3:] = copy.deepcopy(individual)
    global nsga_all_evals 
    nsga_all_evals = np.vstack((nsga_all_evals,this_eval))

    return score, obj2

# Create the NSGA-II algorithm components
creator.create("FitnessMax", base.Fitness, weights=(1.0, -1.0))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_bool", np.random.randint, 2)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=X_train.shape[1])
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluate_individual)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selNSGA2)

In [45]:
# Set the algorithm parameters
population_size = 50
max_generations = num_iters
offspring_size = population_size

# Create the initial population
population = toolbox.population(n=population_size)

# Run the NSGA-II algorithm
for generation in range(max_generations):
    offspring = algorithms.varAnd(population, toolbox, cxpb=0.5, mutpb=0.1)
    fits = toolbox.map(toolbox.evaluate, offspring)
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = fit
    population = toolbox.select(offspring, k=population_size)

nsga_nondom_sols = get_nondom_fronts(nsga_all_evals[1:])
    
print(nsga_nondom_sols[:,:2])

[[ 0.79166667 13.        ]]


In [46]:
np.savetxt(f"{path}/{data_set}_NSGA2.csv", nsga_nondom_sols, delimiter=",")

# Multi Objective Particle Swarm Optimization

In [47]:
mopso_all_evals = np.zeros((1, 3 + total_features))

class MOPSOFeatureSelection:
    def __init__(self, n_particles, n_iterations, w=0.5, c1=1.5, c2=1.5, n_features=total_features):
        self.n_particles = n_particles
        self.n_iterations = n_iterations
        self.w = w # inertia
        self.c1 = c1 # cognitive
        self.c2 = c2 # social
        self.n_features = n_features

    def run(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train
        self.n_features = X_train.shape[1] if self.n_features is None else self.n_features

        self.particles = np.random.randint(0, 2, size=(self.n_particles, self.n_features))
        self.velocities = np.random.uniform(size=(self.n_particles, self.n_features))

        self.pbest = np.copy(self.particles)
        self.gbest = self.get_global_best()

        for _ in range(self.n_iterations):
            self.update_particles()
            self.update_pbest()
            self.update_gbest()
    
    # Sets random individual with highest accuracy as the first globals
    def get_global_best(self):
        scores = [self.evaluate(chromosome)[1] for chromosome in self.pbest]
        gbest_idx = np.argmax(scores)
        return self.pbest[gbest_idx]

    def evaluate(self, chromosome):
        if not any(chromosome):
            # To avoid evaluating an individual with all zeros
            return 0.0, 0.0

        selected_features = np.where(chromosome == 1)[0]
        if len(selected_features) == 0:
            # At least one feature should be selected
            return 0.0, 0.0
        global clf_
        start = process_time()
        clf_.fit(X_train[:, selected_features], y_train)
        end = process_time()
        y_pred = clf_.predict(X_test[:, selected_features])
        
        if metric == 'accuracy':
            accuracy = accuracy_score(y_test, y_pred)
        else:
            accuracy = recall_score(y_test, y_pred)
            
        if objective == 'time':
            obj2 = normalise(end-start)
        else:
            obj2 = len(selected_features)
        
        # For keeping track of all evaluations and doing non-dom sort
        this_eval = np.zeros((1, 3 + self.n_features))
        this_eval[:,:3] = (accuracy, obj2, (1-accuracy))
        this_eval[:,3:] = copy.deepcopy(chromosome)
        global mopso_all_evals 
        mopso_all_evals = np.vstack((mopso_all_evals,this_eval))
        
        return accuracy, obj2

    def update_particles(self):
        r1, r2 = np.random.uniform(size=(2, self.n_particles, self.n_features))
        cognitive_component = self.c1 * r1 * (self.pbest - self.particles)
        social_component = self.c2 * r2 * (self.gbest - self.particles)
        self.velocities = self.w * self.velocities + cognitive_component + social_component
        self.particles = (self.velocities > 0.5).astype(int)

    def update_pbest(self):
        pbest_values = [self.evaluate(chromosome) for chromosome in self.pbest]
        current_values = [self.evaluate(chromosome) for chromosome in self.particles]

        for i, (pbest_value, current_value) in enumerate(zip(pbest_values, current_values)):
            if current_value[0] > pbest_value[0] and current_value[1] < pbest_value[1]:
                self.pbest[i] = np.copy(self.particles[i])

    def update_gbest(self):
        gbest_value = self.evaluate(self.gbest)
        for i in range(self.n_particles):
            current_value = self.evaluate(self.particles[i])
            if current_value[0] > gbest_value[0] and current_value[1] < gbest_value[1]:
                self.gbest = np.copy(self.particles[i])


In [48]:
# Driver code
if __name__ == "__main__":
    n_particles = 20
    n_iterations = num_iters

    MOPSO = MOPSOFeatureSelection(n_particles, n_iterations)
    
    MOPSO.run(X_train, y_train)

    mopso_nondom_sols = get_nondom_fronts(mopso_all_evals[1:])
    
    print(mopso_nondom_sols[:,:2])

[[0.79166667 9.        ]
 [0.79166667 9.        ]
 [0.         8.        ]
 [0.         8.        ]]


In [49]:
np.savetxt(f"{path}/{data_set}_MOPSO.csv", mopso_nondom_sols, delimiter=",")

# Multi Objective Differential Evolution

The main difference between a generic genetic algorithm and a differential evolution algorithm is the way that new solutions are created. In a genetic algorithm, new solutions are created by mutating existing solutions. This can be done by randomly changing the values of some of the genes in the solution, or by swapping genes between two different solutions.

In a differential evolution algorithm, new solutions are created by using a vectorized mutation operator. This operator takes three existing solutions and adds a weighted difference between two of them to the third one. This creates a new solution that is hopefully better than the existing solutions.

The DE_mutate() operator first checks if the mutation rate, cr, is greater than a random number. If it is, then the mutation operator does not mutate the individual. Otherwise, the mutation operator randomly selects three individuals from the population. The difference between two of the individuals is then calculated. The new solution is then created by replacing the genes in the individual with the genes from the difference vector.

In [50]:
de_all_evals = np.zeros((1, 3 + total_features))

def evaluate_individual(individual):
    
    
    selected_features = [bool(bit) for bit in individual]
    X_train_selected = X_train[:, selected_features]
    X_test_selected = X_test[:, selected_features]    
    n_feats = len(np.where(np.asarray(individual) == 1)[0].tolist())
    
    if n_feats == 0:
        return 0,0
        
    start = process_time()
    model = clf_.fit(X_train_selected, y_train)
    end = process_time()
    
    y_pred = model.predict(X_test_selected)
    
    if metric == 'accuracy':
        score = accuracy_score(y_test, y_pred)
    else:
        score = recall_score(y_test, y_pred)
        
    if objective == 'time':
        obj2 = normalise(end-start)
    else:
        obj2 = n_feats
    
    this_eval = np.zeros((1, 3 + total_features))
    this_eval[:,:3] = (score, obj2, (1-score))
    this_eval[:,3:] = copy.deepcopy(individual)
    global de_all_evals 
    de_all_evals = np.vstack((de_all_evals,this_eval))

    return score, obj2

def DE_mutate(individual, population, cr=0.5, F=0.7):
    if random.random() < cr:
        return individual

    a, b, c = random.sample(population, 3)
    diff = np.asarray(b) - np.asarray(a)
    mutant = []
    for i in range(len(individual)):
        if random.random() < F:
            mutant.append(1 if individual[i] == 0 else 0)
        else:
            mutant.append(individual[i])
            
        return mutant

# Create the DEAP toolbox
creator.create("FitnessMax", base.Fitness, weights=(1.0, -1.0))  # Maximizing score, minimizing feat number
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_bool", np.random.randint, 2)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=total_features)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Register the evaluation and mate functions
toolbox.register("evaluate", evaluate_individual)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", DE_mutate)  
toolbox.register("select", tools.selNSGA2)


def main(population_size, generations):
    np.random.seed(42)

    # Initialize population
    population = toolbox.population(n=population_size)

    pareto_fronts = []

    for gen in range(generations):
        # Evaluate individuals
        fitnesses = [toolbox.evaluate(ind) for ind in population]
        for ind, fit in zip(population, fitnesses):
            ind.fitness.values = fit

        # Select the next generation individuals
        offspring = toolbox.select(population, len(population))

        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))
        
        for mutant in offspring:
            toolbox.mutate(mutant, offspring)
            del mutant.fitness.values
            
        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if np.random.rand() < 0.5:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        # Evaluate the offspring
        fitnesses = [toolbox.evaluate(ind) for ind in offspring]
        for ind, fit in zip(offspring, fitnesses):
            ind.fitness.values = fit

        # Replace the current population by the offspring
        population[:] = offspring

    return None

In [51]:
if __name__ == "__main__":
    MODE = main(population_size=100, generations=num_iters)
    de_nondom_sols = get_nondom_fronts(de_all_evals[1:])
    print(de_nondom_sols[:,:2])

[[ 0.79166667 12.        ]]


In [52]:
np.savetxt(f"{path}/{data_set}_MODE.csv", de_nondom_sols, delimiter=",")

## Strength Pareto Evolutionary Algorithm II

SPEA2 and NSGA2 are both multi-objective optimization algorithms that are based on the concept of Pareto dominance. However, there are some key differences between the two algorithms.

SPEA2 uses fitness sharing, while NSGA2 uses crowding distance. Fitness sharing is a technique that penalizes individuals that are too similar to each other. This helps to maintain a diverse population of individuals, which is important for multi-objective optimization. Crowding distance is a measure of how crowded an individual is in the objective space. Individuals with a high crowding distance are more likely to be selected for the next generation, as they are more likely to represent a unique point in the objective space.

SPEA2 uses a weighted sum of the objectives, while NSGA2 uses a Pareto ranking. SPEA2 calculates the fitness of an individual as a weighted sum of the objectives. This means that the algorithm can be biased towards one or more of the objectives. NSGA2, on the other hand, uses a Pareto ranking. This means that the algorithm does not favor any of the objectives, and it tries to find a set of individuals that are all Pareto optimal.

In general, SPEA2 is considered to be more robust than NSGA2. This is because SPEA2 is less likely to get stuck in local optima. However, NSGA2 is generally faster than SPEA2.

This algorithm uses deap's eaMuPlusLambda function, which first creates a new population of solutions by selecting mu parents from the current population and performing crossover on them. The new population is then mutated with probability mutpb. The function then repeats this process until a stopping criterion is met. The stopping criterion can be a maximum number of generations, a maximum time limit, or a certain fitness threshold.

In [53]:
# Define the evaluation function
def evaluate(individual):
    """
    Evaluates an individual by first selecting the features that are turned on in the individual,
    then applying feature selection to the training and test data, and finally training a classifier and calculating the accuracy.
    The function returns the number of selected features and the negative accuracy.
    """

    selected_features = [i for i, val in enumerate(individual) if val]
    if not selected_features:
        return 0, 0,  # Return 0 accuracy and 0 features if none are selected
    
    n_feats = len(np.where(np.asarray(individual) == 1)[0].tolist())

    # Apply feature selection
    selected_X_train = X_train[:, selected_features]
    selected_X_test = X_test[:, selected_features]

    # Train a classifier
    start = process_time()
    model = clf_.fit(selected_X_train, y_train)
    end = process_time()

    y_pred = model.predict(selected_X_test)
    
    if metric == 'accuracy':
        accuracy = accuracy_score(y_test, y_pred)
    else:
        accuracy = recall_score(y_test, y_pred)
        
    if objective == 'time':
        obj2 = normalise(end-start)
    else:
        obj2 = n_feats
        
    # Return negative accuracy to maximize it
    return -accuracy, obj2

# DEAP initialization
creator.create("FitnessMulti", base.Fitness, weights=(-1.0, -1.0))
creator.create("Individual", list, fitness=creator.FitnessMulti)

toolbox = base.Toolbox()
toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=total_features)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Genetic operators
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selSPEA2) # this selection operator makes this an SPEA2
toolbox.register("evaluate", evaluate)


# Set up and run the algorithm
def main():
    """
    Sets up and runs the SPEA-2 algorithm.
    It first creates a random population of individuals,
    then uses the `eaMuPlusLambda()` algorithm to evolve the population over 50 generations.
    The algorithm uses the `cxTwoPoint()` and `mutFlipBit()` genetic operators to create new individuals,
    and the `selSPEA2()` selection operator to select individuals for the next generation.

    The main function then displays the Pareto front solutions, which are the individuals that have the best trade-off between the number of selected features and the accuracy.
    """

    random.seed(42)
    
    pop = toolbox.population(n=50)
    hof = tools.ParetoFront()

    algorithms.eaMuPlusLambda(pop, toolbox, mu=50, lambda_=100, cxpb=0.7, mutpb=0.3, ngen=num_iters, stats=None,
                              halloffame=hof, verbose=False)

    return hof

In [54]:
if __name__ == "__main__":
    
    # Hall of fame is a functionality in DEAP that returns the Pareto fittest individuals
    hof = main()
    
    spea2_nondom_sols = np.zeros((len(hof),3+total_features))
    
    # Write the Pareto front solutions into a matrix
    for i, ind in enumerate(hof):
        
        score = -ind.fitness.values[0]
        
        obj2 = ind.fitness.values[1]
        
        # Write in the same universal results matrix format
        spea2_nondom_sols[i,:3] = (score, obj2, 1-score)
        
        spea2_nondom_sols[i,3:] = ind
        
# Algorithm sometimes returns zeros as results, this gets rid of them
for i, point in enumerate(spea2_nondom_sols):

    if point[0] <=0:
        spea2_nondom_sols = np.delete(spea2_nondom_sols, i, 0)
        
print(spea2_nondom_sols[:,:2])

[[0.79166667 1.        ]]


In [55]:
np.savetxt(f"{path}/{data_set}_SPEA2.csv", spea2_nondom_sols, delimiter=",")

# Parameter Search